In [65]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import StratifiedKFold
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

# data input 
path = 'trainrat_new.txt'
data = pd.read_csv(path,sep=' ')
print(data.head())

# Data extraction,Win
xWin = data[data.label == "W"]
xWin = xWin.assign(W=1,D=0,L=0) 
xWin = xWin.drop("label",axis=1)
xWin = xWin.sample(n=len(xWin))#random sort
yWin = xWin[["W","D","L"]]

# Data extraction,Draw
xDraw = data[data.label == "D"]
xDraw = xDraw.assign(W=0,D=1,L=0) 
xDraw = xDraw.drop("label",axis=1)
xDraw = xDraw.sample(n=len(xDraw))#random sort
yDraw = xDraw[["W","D","L"]]

# Data extraction,Lose
xLose = data[data.label == "L"]
xLose = xLose.assign(W=0,D=0,L=1) 
xLose = xLose.drop("label",axis=1)
xLose = xLose.sample(n=len(xLose))#random sort
yLose = xLose[["W","D","L"]]

# Group data together,use for normal k-fold
xAll = pd.concat([xWin, xDraw, xLose])
xAll = xAll.sample(n=len(xAll))#random sort
yAll = xAll[["W","D","L"]]

# Data drop
xWin = xWin.drop(["W","D","L"],axis=1)
xDraw = xDraw.drop(["W","D","L"],axis=1)
xLose = xLose.drop(["W","D","L"],axis=1)
xAll = xAll.drop(["W","D","L"],axis=1)

# Data separate and making dataset
xWin_separate = []
xDraw_separate = []
xLose_separate = []
xAll_separate = []
yWin_separate = []
yDraw_separate = []
yLose_separate = []
yAll_separate = []
x_separate = []
y_separate = []
for i in range(10):
    xWin_separate.append(xWin[i::10])
    xDraw_separate.append(xDraw[i::10])
    xLose_separate.append(xLose[i::10])
    xAll_separate.append(xAll[i::10])
    yWin_separate.append(yWin[i::10])
    yDraw_separate.append(yDraw[i::10])
    yLose_separate.append(yLose[i::10])
    yAll_separate.append(yAll[i::10])
    # merge for stratified sampling
    x_separate.append(pd.concat([xWin_separate[i],xDraw_separate[i],xLose_separate[i]]))
    y_separate.append(pd.concat([yWin_separate[i],yDraw_separate[i],yLose_separate[i]]))
    # assign a number to make final input data
    x_separate[i] = x_separate[i].assign(separate_num=i)
    y_separate[i] = y_separate[i].assign(separate_num=i)
    xAll_separate[i] = xAll_separate[i].assign(separate_num=i)
    yAll_separate[i] = yAll_separate[i].assign(separate_num=i)

# integrate everything once
x_separate_merge = x_separate[0]
y_separate_merge = y_separate[0]
xAll_separate_merge = xAll_separate[0]
yAll_separate_merge = yAll_separate[0]

# 
for i in range(1,10):
    x_separate_merge = x_separate_merge.append(x_separate[i])
    y_separate_merge = y_separate_merge.append(y_separate[i])
    xAll_separate_merge = xAll_separate_merge.append(xAll_separate[i])
    yAll_separate_merge = yAll_separate_merge.append(yAll_separate[i])
    

# make final input data
x_train = []
y_train = []
x_test = []
y_test = []
xAll_train = []
yAll_train = []
xAll_test = []
yAll_test = []
for i in range(10):
    x_train.append(x_separate_merge[x_separate_merge['separate_num'] != i])
    y_train.append(y_separate_merge[y_separate_merge['separate_num'] != i])
    x_test.append(x_separate_merge[x_separate_merge['separate_num'] == i])
    y_test.append(y_separate_merge[y_separate_merge['separate_num'] == i])
    xAll_train.append(xAll_separate_merge[xAll_separate_merge['separate_num'] != i])
    yAll_train.append(yAll_separate_merge[yAll_separate_merge['separate_num'] != i])
    xAll_test.append(xAll_separate_merge[xAll_separate_merge['separate_num'] == i])
    yAll_test.append(yAll_separate_merge[yAll_separate_merge['separate_num'] == i])
    # delete separate_num
    x_train[i] = x_train[i].drop(['separate_num'],axis=1)
    x_test[i] = x_test[i].drop(['separate_num'],axis=1)
    y_train[i] = y_train[i].drop(['separate_num'],axis=1)
    y_test[i] = y_test[i].drop(['separate_num'],axis=1)
    xAll_train[i] = xAll_train[i].drop(['separate_num'],axis=1)
    xAll_test[i] = xAll_test[i].drop(['separate_num'],axis=1)
    yAll_train[i] = yAll_train[i].drop(['separate_num'],axis=1)
    yAll_test[i] = yAll_test[i].drop(['separate_num'],axis=1)


print(x_train[0].head())
print(y_train[0].head())
print(xAll_train[0].head())
print(yAll_test[0].head())

print(len(x_train[0]))
print(len(y_test[0]))
print(len(xAll_train[0]))
print(len(yAll_test[0]))



      label     HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF  \
13680     D -0.013286  0.055444 -0.032403  0.112291 -0.013286  0.163329   
2729      L  1.006109  0.230130  0.488624  0.489483  0.985480  0.083888   
26413     D -0.034506  0.105019  0.012418  0.435008 -0.061290  0.057348   
8975      D  4.744033 -1.088280  1.720375 -1.439166  0.698823  0.749711   
13324     W  0.016564 -0.053097  0.040713  0.194267  0.026022  0.175712   

          AAATT     AADEF  
13680 -0.032403 -0.015245  
2729   0.432996 -0.060702  
26413  0.395053  0.766563  
8975   0.396043  1.203952  
13324  0.040616  0.915641  
          HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF     AAATT  \
9221  -0.053163 -0.170218  0.061345  0.130607 -0.244653  1.491166  0.168156   
9929   1.873492  0.014206  0.407465 -0.709762 -1.779173  0.074977 -0.209977   
29197  0.347706 -0.101115 -0.434594 -0.109751 -0.054333  0.319899 -0.028997   
290    1.100914  0.016453  0.482952 -0.289089  1.289542  0.312502

In [25]:
# create model
modelArelu = Sequential()
modelArelu.add(Dense(10, activation='relu'))
modelArelu.add(Dense(10, activation='relu'))
modelArelu.add(Dense(1, activation='sigmoid'))


def mean_squared_error(y_true, y_pred):
    return 

modelArelu.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [26]:
cvscores = []
batch_size = 16
num_classes = 10
epochs = 10
for train, test in kfold.split(x, y):
    print("TRAIN:", train, "TEST:", test)
    X_train, X_test = x.iloc[train], x.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]

    # Fit the model
    modelArelu.fit(X_train.values, y_train.values,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1
                  )


    results = modelArelu.predict(X_test, batch_size=batch_size, verbose=0, steps=None)
    
    print(results)
    print(y_test.values)


TRAIN: [    0     1     3 ... 28183 28184 28185] TEST: [    2     8    17 ... 28166 28173 28176]
Epoch 1/10
25367/25367 [==============================] - 1s 37us/step - loss: 0.6569 - acc: 0.6059
Epoch 2/10
25367/25367 [==============================] - 1s 31us/step - loss: 0.6515 - acc: 0.6147
Epoch 3/10
25367/25367 [==============================] - 1s 30us/step - loss: 0.6509 - acc: 0.6170
Epoch 4/10
25367/25367 [==============================] - 1s 30us/step - loss: 0.6506 - acc: 0.6153
Epoch 5/10
25367/25367 [==============================] - 1s 30us/step - loss: 0.6503 - acc: 0.6142
Epoch 6/10
25367/25367 [==============================] - 1s 31us/step - loss: 0.6499 - acc: 0.6164
Epoch 7/10
25367/25367 [==============================] - 1s 31us/step - loss: 0.6500 - acc: 0.6158
Epoch 8/10
25367/25367 [==============================] - 1s 31us/step - loss: 0.6495 - acc: 0.6173
Epoch 9/10
25367/25367 [==============================] - 1s 30us/step - loss: 0.6498 - acc: 0.6157
Epo

In [1]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from sklearn.model_selection import StratifiedKFold
import numpy

fold_num = 10
batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


11493376/11490434 [==============================] - 3s 0us/step
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)          